<a href="https://colab.research.google.com/github/gingerchien/QuantHub/blob/main/vectorbt_for_beginners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a compilation of **Chad Thackray's Vectorbt for beginners - Full Python Course** located below and a few of my ideas after learning the course:

https://www.youtube.com/watch?v=JOdEZMcvyac&list=PLKCjdQRzJEHzUc09EhgHD_to93yLnjT0P


#What is Vectorbt?

Accodring to their webstite: https://vectorbt.dev/
vectorbt is a Python package for quantitative analysis that takes a novel approach to backtesting: it operates entirely on pandas and NumPy objects, and is accelerated by Numba to analyze any data at speed and scale. This allows for testing of many thousands of strategies in seconds.

In contrast to other backtesters, vectorbt represents complex data as (structured) NumPy arrays. This enables superfast computation using vectorized operations with NumPy and non-vectorized but dynamically compiled operations with Numba. It also integrates Plotly and Jupyter Widgets to display complex charts and dashboards akin to Tableau right in the Jupyter notebook. Due to high performance, vectorbt can process large amounts of data even without GPU and parallelization and enables the user to interact with data-hungry widgets without significant delays.

# 0. Quickstart

* Documentation: https://vectorbt.dev/api/indicators/basic/
* Github: https://github.com/polakowo/vectorbt

In [6]:
#check python version
!python3 --version

Python 3.10.12


## Install Libraries

In [7]:
#install vectorbt
#!pip3 install vectorbt

#install Yahoo Finance
#!pip3 install yfinance

In [8]:
import vectorbt as vbt
import yfinance as yf
import datetime

## Download Data

In [9]:
end_date = datetime.datetime.now()
start_date= end_date - datetime.timedelta(days=7)
print(start_date)

#btc_price = vbt.YFData.download(['BTC-USD', 'ETH-USD'], interval='1m', start=start_date, end=end_date ,missing_index='drop').get('Close')
btc_price = vbt.YFData.download('BTC-USD', missing_index='drop').get('Close')

2023-12-14 06:15:29.112487


In [10]:
print(btc_price.tail())
print(btc_price.head())
print(type(btc_price))

Date
2023-12-16 00:00:00+00:00    42240.117188
2023-12-17 00:00:00+00:00    41364.664062
2023-12-18 00:00:00+00:00    42623.539062
2023-12-19 00:00:00+00:00    42270.527344
2023-12-21 00:00:00+00:00    43498.203125
Name: Close, dtype: float64
Date
2014-09-17 00:00:00+00:00    457.334015
2014-09-18 00:00:00+00:00    424.440002
2014-09-19 00:00:00+00:00    394.795990
2014-09-20 00:00:00+00:00    408.903992
2014-09-21 00:00:00+00:00    398.821014
Name: Close, dtype: float64
<class 'pandas.core.series.Series'>


## Technical Indicators

In [11]:
#rsi = vbt.RSI.run(btc_price, window=[14,21]) different windows
rsi = vbt.RSI.run(btc_price, window=14)
print(rsi.rsi) #to extract the value, call rsi.rsi

Date
2014-09-17 00:00:00+00:00          NaN
2014-09-18 00:00:00+00:00          NaN
2014-09-19 00:00:00+00:00          NaN
2014-09-20 00:00:00+00:00          NaN
2014-09-21 00:00:00+00:00          NaN
                               ...    
2023-12-16 00:00:00+00:00    61.071835
2023-12-17 00:00:00+00:00    55.392152
2023-12-18 00:00:00+00:00    52.656277
2023-12-19 00:00:00+00:00    41.267414
2023-12-21 00:00:00+00:00    48.897451
Name: (14, Close), Length: 3382, dtype: float64


### Turning Indicator into True/Falst Signals for processing with Vectorbt

In [12]:
entries = rsi.rsi_crossed_below(30)
#print(entries.to_string())

In [13]:
exits = rsi.rsi_crossed_above(70)
#print(exits.to_string())

In [14]:
#test with portfolio pf
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

#check what's going on
print(pf.stats())

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2014-09-17 00:00:00+00:00
End                           2023-12-21 00:00:00+00:00
Period                                             3382
Start Value                                       100.0
End Value                                     79.460413
Total Return [%]                             -20.539587
Benchmark Return [%]                         9411.25473
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              89.870408
Max Drawdown Duration                            2174.0
Total Trades                                         37
Total Closed Trades                                  37
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  59.459459
Best Trade [%]                                37.937622
Worst Trade [%]                              -43

In [15]:
print(pf.total_return()) #can extract individual items from stats()

-0.2053958686215688


In [16]:
#plotting results, the most simple plot in vectorbt
pf.plot().show()

# 1. Custom Indicators using the Indicator Factory
* Reference: https://vectorbt.dev/api/indicators/factory/#naive-approach

In [17]:
end_time = datetime.datetime.now()
start_time= end_time - datetime.timedelta(days=3)
btc_price = vbt.YFData.download('BTC-USD', missing_index='drop', start= start_time, end=end_time, interval='1m').get('Close')

In [18]:
print(btc_price)

Datetime
2023-12-18 06:16:00+00:00    41025.171875
2023-12-18 06:17:00+00:00    41018.605469
2023-12-18 06:18:00+00:00    41058.542969
2023-12-18 06:19:00+00:00    41065.925781
2023-12-18 06:20:00+00:00    41067.144531
                                 ...     
2023-12-21 06:10:00+00:00    43552.957031
2023-12-21 06:11:00+00:00    43546.195312
2023-12-21 06:12:00+00:00    43538.128906
2023-12-21 06:13:00+00:00    43510.937500
2023-12-21 06:14:00+00:00    43498.203125
Name: Close, Length: 4178, dtype: float64


## Define Custom Indicator

In [19]:
import numpy as np
def custom_indicator(close, rsi_window=14, ma_window=50):
  rsi = vbt.RSI.run(close, window=rsi_window).rsi.to_numpy() #the .rsi grabs the actual values to turn it into signal later as raw values are not helpful
  ma = vbt.MA.run(close, window=ma_window).ma.to_numpy()#converting to numpy arrays
  #print(rsi)
  #print(ma)
  #create signal
  trend = np.where(rsi>70, -1, 0)
  trend = np.where((rsi<30) & (close<ma) , 1, trend) #only use & version here, AND does not work, for or use |
  #print(trend)
  return trend

#create indicator object so Vectorbt can recognize it for any custom indicators
indicator = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window'],
    output_names=['value']
).from_apply_func(custom_indicator, rsi_window=14, ma_window=50) #provide default values then point it to the custom indictor

#run and get indicator values
res = indicator.run(btc_price, rsi_window=21, ma_window=50)
#print(res.value)
entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

Start                         2023-12-18 06:16:00+00:00
End                           2023-12-21 06:14:00+00:00
Period                                  2 days 21:38:00
Start Value                                       100.0
End Value                                    101.793365
Total Return [%]                               1.793365
Benchmark Return [%]                           6.028083
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               2.841064
Max Drawdown Duration                   1 days 04:48:00
Total Trades                                         30
Total Closed Trades                                  29
Total Open Trades                                     1
Open Trade PnL                                -0.150246
Win Rate [%]                                  79.310345
Best Trade [%]                                 0.542534
Worst Trade [%]                               -1

In [20]:
pf.plot().show()

## What if we want a 5 min indicator on the 1 min data?

In [29]:
import numpy as np

end_time = datetime.datetime.now()
start_time= end_time - datetime.timedelta(days=3)
btc_price = vbt.YFData.download(['BTC-USD', 'ETH-USD'], missing_index='drop', start= start_time, end=end_time, interval='1m').get('Close')

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html
# broadcast_axis = 0 , interchangeable w/ 'index', join = 'right' (takes the keys from the right table only (close is the right table, rsi is the left table))
def custom_indicator2(close, rsi_window=14, ma_window=50):
  close_5m = close.resample('5T').last() #converting 1m candles to 5m
  rsi = vbt.RSI.run(close_5m, window=rsi_window).rsi
  rsi, _ = rsi.align(close, broadcast_axis=0, method='ffill', join='right') #this makes the rsi align to the 1 min but at 5 min intervals, where the other 4 min are NAN in between. forward fill helps fill in the gap.

  #convert everything to numpy
  close = close.to_numpy()
  rsi = rsi.to_numpy()

  ma = vbt.MA.run(close, window=ma_window).ma.to_numpy()
  #print(rsi)
  #print(ma)
  #create signal
  trend = np.where(rsi>70, -1, 0)
  trend = np.where((rsi<30) & (close<ma) , 1, trend) #only use & version here, AND does not work, for or use |
  #print(trend)
  return trend

#create indicator object so Vectorbt can recognize it for any custom indicators
indicator2 = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window'],
    output_names=['value']
).from_apply_func(custom_indicator2, rsi_window=14, ma_window=50, keep_pd=True) #keep_pd = True, this ensures everything stays in panda format instead of getting turned into numpy arrays

#run and get indicator values
res = indicator2.run(btc_price, rsi_window=21, ma_window=50)
#print(res.value)
entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

/usr/local/lib/python3.10/dist-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.



Start                         2023-12-18 06:30:00+00:00
End                           2023-12-21 06:27:00+00:00
Period                                  2 days 20:24:00
Start Value                                       100.0
End Value                                     98.032599
Total Return [%]                              -1.967401
Benchmark Return [%]                           3.627121
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               4.230463
Max Drawdown Duration                   2 days 16:19:00
Total Trades                                        3.5
Total Closed Trades                                 3.5
Total Open Trades                                   0.0
Open Trade PnL                                      0.0
Win Rate [%]                                       25.0
Best Trade [%]                                 0.379372
Worst Trade [%]                               -1

<ipython-input-29-13e14f1b6278>:44: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x79a16d6fdcf0>. Pass column to select a single column/group.



# 2. Hyperparameter Optimization

In [30]:
end_time = datetime.datetime.now()
start_time= end_time - datetime.timedelta(days=3)
btc_price = vbt.YFData.download(['BTC-USD', 'ETH-USD'], missing_index='drop', start= start_time, end=end_time, interval='1m').get('Close')

def custom_indicator2(close, rsi_window=14, ma_window=50, entry = 30, exit=70):
  close_5m = close.resample('5T').last()
  rsi = vbt.RSI.run(close_5m, window=rsi_window).rsi
  rsi, _ = rsi.align(close, broadcast_axis=0, method='ffill', join='right')

  #convert everything to numpy
  close = close.to_numpy()
  rsi = rsi.to_numpy()

  ma = vbt.MA.run(close, window=ma_window).ma.to_numpy()
  #print(rsi)
  #print(ma)
  #create signal
  trend = np.where(rsi>exit, -1, 0)
  trend = np.where((rsi<entry) & (close<ma) , 1, trend)
  #print(trend)
  return trend

#create indicator object so Vectorbt can recognize it for any custom indicators
indicator2 = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window', 'entry', 'exit'],
    output_names=['value']
).from_apply_func(custom_indicator2, rsi_window=14, ma_window=50, entry=30, exit=70, keep_pd=True)

#run and get indicator values
res = indicator2.run(btc_price, rsi_window=np.arange(10,40,step=3, dtype=int), ma_window=np.arange(20,200,step=15, dtype=int), entry=np.arange(10,40,step=4, dtype=int),exit=np.arange(60,85,step=4, dtype=int), param_product=True) #param_product = True shows a combination of all the parameters

entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
returns = pf.total_return()#print out everything by using .to_string()
print(returns.max(), returns.idxmax()) #print out the max combination of returns

print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

/usr/local/lib/python3.10/dist-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.



0.07305935897589862 (28, 20, 34, 84, 'BTC-USD')
Start                         2023-12-18 06:30:00+00:00
End                           2023-12-21 06:27:00+00:00
Period                                  2 days 20:24:00
Start Value                                       100.0
End Value                                     99.751912
Total Return [%]                              -0.248088
Benchmark Return [%]                           3.627121
Max Gross Exposure [%]                             75.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               3.359947
Max Drawdown Duration         1 days 15:03:44.434523809
Total Trades                                   3.540625
Total Closed Trades                            3.122768
Total Open Trades                              0.417857
Open Trade PnL                                 0.057505
Win Rate [%]                                  58.369086
Best Trade [%]                                 0.706572


<ipython-input-30-05194156e690>:43: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x79a16d6fdcf0>. Pass column to select a single column/group.



## To show the other columns, call it out specifically

In [31]:
# print(returns.to_string()) #print out everything by using .to_string()
# returns = returns[returns.index.isin(['ETH-USD'], level='symbol')]

# print(returns.max(), returns.idxmax()) #print out the max combination of returns
# #print(returns.to_string()) #print out everything by using .to_string()
# print(pf.stats()) #if vectorbt could not parse the frequency of the close, it will not return any duration in time units or metrics that requires annualization and throw a bunch of warnings

## Creating Heatmaps

In [32]:
#plotly is used in vectorbt, x_level and y_level can be any parameters
fig = returns.vbt.heatmap(x_level='comb_rsi_window', y_level='comb_entry',
                          slider_level='comb_ma_window')#adding the slider, can use it to sort symbol best combo
fig.show()




## Groupby Axis to Aggregate Parameters if A Lot of Parameters

In [35]:
#Give us a heatmap grouped by the exit and entry values, mean as aggregation function
#for each entry and exit path, it will average the result of parameters
#This will give us only 3 columns and everything else is averaged out
returns = returns.groupby(level=['comb_exit', 'comb_entry', 'symbol']).mean()
print(returns.to_string())
print(returns.max())
print(returns.idxmax())

comb_exit  comb_entry  symbol 
60         10          BTC-USD    0.001192
                       ETH-USD   -0.000306
           14          BTC-USD    0.000322
                       ETH-USD   -0.001467
           18          BTC-USD    0.000846
                       ETH-USD   -0.004603
           22          BTC-USD    0.000874
                       ETH-USD   -0.020635
           26          BTC-USD   -0.006262
                       ETH-USD   -0.027175
           30          BTC-USD   -0.008889
                       ETH-USD   -0.031140
           34          BTC-USD   -0.006540
                       ETH-USD   -0.040379
           38          BTC-USD   -0.003053
                       ETH-USD   -0.041446
64         10          BTC-USD    0.001807
                       ETH-USD    0.000197
           14          BTC-USD    0.001114
                       ETH-USD   -0.001762
           18          BTC-USD    0.002008
                       ETH-USD   -0.004603
           22          

In [37]:
fig = returns.vbt.heatmap(
    x_level='comb_entry',
    y_level='comb_exit',
    slider_level='symbol',
)
fig.show()

## Create Volume Graph with Multiple Parameters

In [42]:
#create indicator object so Vectorbt can recognize it for any custom indicators
indicator2 = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window', 'ma_window', 'entry', 'exit'],
    output_names=['value']
).from_apply_func(custom_indicator2, rsi_window=14, ma_window=50, entry=30, exit=70, keep_pd=True)

#run and get indicator values
res = indicator2.run(btc_price, rsi_window=np.arange(10,40,step=3, dtype=int),
                     #ma_window=np.arange(20,200,step=15, dtype=int),
                     entry=np.arange(10,40,step=4, dtype=int),
                     exit=np.arange(60,85,step=4, dtype=int), param_product=True) #param_product = True shows a combination of all the parameters

entries = res.value == 1.0
exits = res.value == -1.0

# https://vectorbt.dev/api/portfolio/base/#custom-metrics
pf = vbt.Portfolio.from_signals(btc_price, entries, exits, seed=42, freq='m')
returns = pf.total_return()#print out everything by using .to_string()
print(returns.max(), returns.idxmax()) #print out the max combination of returns

fig = returns.vbt.volume(
    x_level='comb_entry',
    y_level='comb_exit',
    z_level='comb_rsi_window',
    slider_level='symbol',
)

fig.show()

0.07354614340782732 (28, 34, 84, 'BTC-USD')


# 3. Optimization Techniques

# 4. Graphing/Dashboarding

# 5. Order Types

# 6. Avoid Over-fitting